#### Importing the libs

In [1]:
import pandas as pd
import os
import numpy as np
import csv

import utils

from joblib import Parallel, delayed

#### Loading the data

In [2]:
train_data = pd.read_csv("./Data/train.csv")
metadata_data = pd.read_csv("./Data/item_metadata.csv")

In [3]:
train_data.head(5)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [4]:
metadata_data.head(5)

,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
2,5834,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...
3,5910,Satellite TV|Sailing|Cosmetic Mirror|Telephone...
4,6066,Satellite TV|Sailing|Diving|Cosmetic Mirror|Sa...


## Process metadata properties into columns

In [5]:
metadata_data = utils.process_attributes(metadata_data)

metadata_data.head(5)

properties,item_id,1 Star,2 Star,3 Star,4 Star,5 Star,Accessible Hotel,Accessible Parking,Adults Only,Air Conditioning,...,Terrace (Hotel),Theme Hotel,Towels,Very Good Rating,Volleyball,Washing Machine,Water Slide,Wheelchair Accessible,WiFi (Public Areas),WiFi (Rooms)
0,5001,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
1,5002,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
2,5003,0,0,0,1,0,1,0,0,1,...,1,0,1,1,0,0,0,1,1,1
3,5004,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,1,1,1
4,5005,0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


## Create data to user lattice

#### Obtain different users

In [6]:
users = train_data["user_id"].unique().tolist()
len(users)

# only a hundredth to test
users = users[:int(len(users) * 0.01)]

len(users)

7308

#### Obtain different Hotels

In [12]:
hotels = metadata_data["item_id"].tolist()

# only a hundredth to test
#hotels = hotels[:int(len(hotels) * 0.01)]

len(hotels)

927142

#### Create the relation matrix

In [13]:
path = f'../ProcessedData/metadata/user_lattice'

os.makedirs(path, exist_ok=True)

with open(f'{path}/user_lattice-relations.csv', 'w') as f:
    write = csv.writer(f, lineterminator='\n')
    for user in users:

        user_actions = train_data[train_data["user_id"] == user]
        clicked_elements = user_actions[user_actions["reference"].str.isnumeric()]["reference"].tolist()

        # Array with ids that will be 1
        user_relation_ids = Parallel(n_jobs=10)(
            delayed(lambda elem: hotels.index(int(elem)) if int(elem) in hotels else None)(clicked_element) for
            clicked_element in clicked_elements)

        user_relation_ids = set(filter(None, user_relation_ids))

        if users.index(user) % 100 == 0:
            print(users.index(user))

        write.writerow(user_relation_ids)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300


In [14]:
with open(f'{path}/user_lattice-objects.csv', 'w') as f:
    write = csv.writer(f, lineterminator='\n')
    write.writerow(users)

In [15]:
with open(f'{path}/user_lattice-attributes.csv', 'w') as f:
    write = csv.writer(f, lineterminator='\n')
    write.writerow(hotels)